In [23]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import cvxpy as cvx
import osqp
from time import time

import mosek

In [24]:
#define sizes of problems
ratio = 5 
K = [5*u for u in range(1,21)]
nPb = len(K) #number of problems

In [25]:
#define time of problems
Tscs = []
Tecos = []
Tosqps = []
Tosqp = []

In [26]:
for k in K:
    
    #define size
    n = ratio*k
    
    # Generate problem data
    sp.random.seed(1)
    F     = sparse.random(n, k, density=0.7, format='csc')
    D     = sparse.diags(np.random.rand(n) * np.sqrt(k), format='csc')
    Sigma = F*F.T + D
    mu    = np.random.randn(n)
    gamma = 1
    
    # Define problem
    x = cvx.Variable(n)
    objective = - mu.T*x + gamma*cvx.quad_form(x,Sigma)
    constraints = [sum(x) == 1, x >= 0]
    
    # Solve SCS
    t = time()
    cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = 'SCS')
    Tscs.append(time() - t)
    
    # Solve ECOS
    t = time()
    cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = 'ECOS')
    Tecos.append(time() - t)
    
    # Solve OSQPsolver
    t = time()
    cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = cvx.OSQP)
    Tosqps.append(time() - t)
    
    #basic OSQP python
    # OSQP data
    P = sparse.block_diag((D, sparse.eye(k)), format='csc')
    q = np.hstack([-mu / (2*gamma), np.zeros(k)])
    A = sparse.vstack([
            sparse.hstack([F.T, -sparse.eye(k)]),
            sparse.hstack([sparse.csc_matrix(np.ones((1, n))), sparse.csc_matrix((1, k))]),
            sparse.hstack((sparse.eye(n), sparse.csc_matrix((n, k))))
        ]).tocsc()
    l = np.hstack([np.zeros(k), 1., np.zeros(n)])
    u = np.hstack([np.zeros(k), 1., np.ones(n)])
    
    # Create an OSQP object
    prob = osqp.OSQP()

    # Setup workspace
    prob.setup(P, q, A, l, u)

    # Solve problem
    t = time()
    res = prob.solve()
    Tosqp.append(time() - t)
    
    
    





-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 30, constraints m = 31
          nnz(P) + nnz(A) = 172
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off

iter   objective    pri res    dua res    rho        time
   1  -3.1174e+00   2.32e+00   8.22e+01   1.00e-01   1.50e-04s
  50  -4.8171e-01   8.75e-06   9.11e-06   1.22e+00   8.58e-05s

status:               solved
number of iterations: 50
optimal o

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 270, constraints m = 271
          nnz(P) + nnz(A) = 7852
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off

iter   objective    pri res    dua res    rho        time
   1  -1.0564e+01   1.82e+00   1.67e+02   1.00e-01   4.02e-03s
 150   1.1085e+00   3.23e-04   5.31e-04   1.76e+00   5.23e-03s

status:               solved
number of iterations: 150
optim

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 510, constraints m = 511
          nnz(P) + nnz(A) = 26732
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off

iter   objective    pri res    dua res    rho        time
   1  -1.0183e+01   1.13e+00   8.99e+01   1.00e-01   1.91e-02s
 200   7.7375e-01   3.88e-02   8.41e-05   1.00e-01   3.31e-02s
 275   3.1189e+00   4.40e-04   8.98e-04   3.20e+00   2.55e-0

In [35]:
Rscs   = []
Recos  = []
Rospqs = []
Rosqp  = []

In [36]:
for i in range(len(Tscs)):
    tscs   = Tscs[i]
    tecos  = Tecos[i]
    tosqps = Tosqps[i]
    tosqp  = Tosqp[i]
    
    mint = min(tscs,tecos,tosqps,tosqp)
    
    Rscs.append(tscs/mint)
    Recos.append(tecos/mint)
    Rospqs.append(tosqps/mint)
    Rospq.append(tosqp/mint)
    

NameError: name 'Rospq' is not defined

1
